In [ ]:
%matplotlib inline

from collections import defaultdict
from IPython import display
from PIL import Image
from torch import nn
from torch.autograd import Variable
from torchvision import models, transforms

import json
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import torch

# Data Acquisition

For this assignment, you must download the dataset provided as a separate link on the course webpage and extract it into `data/`. The dataset contains approximately 20K training images and 100 validation images, with multiple captions/tags for each image. For this assignment, we are only concerned with the tags and ignore the captions.

For question two on the assignment, the dataset also contains a JSON file that maps from the ImageNet labels to the category names. 

Following the data downloading and unzipping, the code below loads in the data into memory accordingly.

In [ ]:
# Define a global transformer to appropriately scale images and subsequently convert them to a Tensor.
img_size = 224
loader = transforms.Compose([
  transforms.Resize(img_size),
  transforms.CenterCrop(img_size),
  transforms.ToTensor(),
]) 
def load_image(filename, volatile=False):
    """
    Simple function to load and preprocess the image.

    1. Open the image.
    2. Scale/crop it and convert it to a float tensor.
    3. Convert it to a variable (all inputs to PyTorch models must be variables).
    4. Add another dimension to the start of the Tensor (b/c VGG expects a batch).
    5. Move the variable onto the GPU.
    """
    image = Image.open(filename).convert('RGB')
    image_tensor = loader(image).float()
    image_var = Variable(image_tensor, volatile=volatile).unsqueeze(0)
    return image_var.cuda()

load_image('data/train2014/COCO_train2014_000000000009.jpg')

In [ ]:
# Load ImageNet label to category name mapping.
imagenet_categories = [value for key,value in sorted(json.load(open('data/imagenet_categories.json')).items(), key=lambda t: int(t[0]))]

# Load annotations file for the 100K training images.
mscoco_train = json.load(open('data/annotations/train2014.json'))
train_ids = [entry['id'] for entry in mscoco_train['images']]
train_id_to_file = {entry['id']: 'data/train2014/' + entry['file_name'] for entry in mscoco_train['images']}
category_to_name = {entry['id']: entry['name'] for entry in mscoco_train['categories']}
category_idx_to_name = [entry['name'] for entry in mscoco_train['categories']]
category_to_idx = {entry['id']: i for i,entry in enumerate(mscoco_train['categories'])}

# Load annotations file for the 100 validation images.
mscoco_val = json.load(open('data/annotations/val2014.json'))
val_ids = [entry['id'] for entry in mscoco_val['images']]
val_id_to_file = {entry['id']: 'data/val2014/' + entry['file_name'] for entry in mscoco_val['images']}

# We extract out all of the category labels for the images in the training set. We use a set to ignore 
# duplicate labels.
train_id_to_categories = defaultdict(set)
for entry in mscoco_train['annotations']:
    train_id_to_categories[entry['image_id']].add(entry['category_id'])

# We extract out all of the category labels for the images in the validation set. We use a set to ignore 
# duplicate labels.
val_id_to_categories = defaultdict(set)
for entry in mscoco_val['annotations']:
    val_id_to_categories[entry['image_id']].add(entry['category_id'])

Let us take a look at an image and its corresponding category labels. We consider the image with the id 391895 and the corresponding filename, `data/val2014/COCO_val2014_000000391895.jpg`. The image is shown below.

![image](data/val2014/COCO_val2014_000000391895.jpg)

The following code determines the category labels for this image.

In [ ]:
for i,category in enumerate(val_id_to_categories[391895]):
    print("%d. %s" % (i, category_to_name[category]))

# 1. Loading a Pre-trained Convolutional Neural Network (CNN)

We will work with the VGG-16 image classification CNN network first introduced in [Very Deep Convolutional Neural Networks for Large-Scale Image Recognition](https://arxiv.org/pdf/1409.1556.pdf) by K. Simonyan and A. Zisserman.

Fairly straightforwardly, we load the pre-trained VGG model and indicate to PyTorch that we are using the model for inference rather than training.

In [ ]:
vgg_model = models.vgg16(pretrained=True)
vgg_model = vgg_model.cuda()

# Let's see what the model looks like.
vgg_model.eval()

# 2. Making Predictions Using VGG-16

Given the pre-trained network, we must now write the code to make predictions on the 100 validation images via a forward pass through the network. Typically the final layer of VGG-16 is a softmax layer, however the pre-trained PyTorch model that we are using does not have softmax built into the final layer (instead opting to incorporate it into the loss function) and therefore we must **manually** apply softmax to the output of the function.

In [ ]:
softmax = nn.Softmax(dim=1)

for image_id in val_ids[:10]:
    # Display the image.
    display.display(display.Image(val_id_to_file[image_id]))

    # Print all of the category labels for this image.
    print("Ground Truth Labels:")
    for i,category in enumerate(val_id_to_categories[image_id]):
        print("%d. %s" % (i, category_to_name[category]))
  
    # Load/preprocess the image.
    img = load_image(val_id_to_file[image_id])

    # Run the image through the model and softmax.
    label_likelihoods = softmax(vgg_model(img)).squeeze()

    # Get the top 5 labels, and their corresponding likelihoods.
    probs, indices = label_likelihoods.topk(5)

    # Iterate and print out the predictions.
    print("Predictions:")
    for i in range(5):
        print("%d. %s (%.3f)" % (i, imagenet_categories[indices.data[i]], probs.data[i]))

# 3. Computing Generic Visual Features using CNN

Since, rather than the output of VGG, we want a fixed sized vector representation of each image, we remove the last linear layer. The implementation of the forward function for VGG is shown below:

```
x = self.features(x)
x = x.view(x.size(0), -1)
x = self.classifier(x)
```
We aim to preserve everything but the final component of the classifier, meaning we must define an alternative equivalent to `self.classifier`.

In [ ]:
# Remove the final layer of the classifier, and indicate to PyTorch that the model is being used for inference
# rather than training (most importantly, this disables dropout).
modified_classifier = nn.Sequential(*list(vgg_model.classifier.children())[:-1])
modified_classifier.eval()

In [ ]:
# First we vectorize all of the training images and write the results to a file.
# TODO: Do more than just the first 500 (don't want to run everything right now)
training_vectors = []
for i,image_id in enumerate(train_ids):
    if (i%1000 == 0):
        print(i)
    # Load/preprocess the image.
    img = load_image(train_id_to_file[image_id])

    # Run through the convolutional layers and resize the output.
    features_output = vgg_model.features(img)
    classifier_input = features_output.view(1, -1)

    # Run through all but final classifier layers.
    output = modified_classifier(classifier_input)
    training_vectors.append(np.array(list(output.data.squeeze())))

# For simplicity, we convert this to a numpy array and save the result to a file.
training_vectors = np.stack(training_vectors, axis=0)
np.save(open('outputs/training_vectors', 'wb+'), training_vectors)

In [ ]:
import pickle
training_vectors = pickle.load(open("outputs/training_vectors.p","rb"))
validation_vectors = np.load("./outputs/validation_vectors")

In [ ]:
# Next we vectorize all of the validation images and write the results to a file.
validation_vectors = []
for image_id in val_ids:
    # Load/preprocess the image.
    img = load_image(val_id_to_file[image_id])

    # Run through the convolutional layers and resize the output.
    features_output = vgg_model.features(img)
    classifier_input = features_output.view(1, -1)

    # Run through all but final classifier layers.
    output = modified_classifier(classifier_input)
    validation_vectors.append(list(output.data.squeeze()))

# For simplicity, we convert this to a numpy array and save the result to a file.
validation_vectors = np.array(validation_vectors)
np.save(open('outputs/validation_vectors', 'wb+'), validation_vectors)

# 4. Visual Similarity

We now use the generated vectors, to find the closest training image for every validation image. This can easily be done by finding the training vector that minimizes the Euclidean distance for every validation image. We repeat this exercise for the first 10 images in the validation set.

In [ ]:
for i,(image_id,vector) in enumerate(zip(val_ids, validation_vectors)):
    print("Processing image %d" % i)  
    
    # Identify the index of the closest training vector.
    closest_idx = min(range(len(training_vectors)), key=lambda i: np.linalg.norm(training_vectors[i] - vector))

    # Show the two images, first the original and then the closest training.
    display.display(display.Image(val_id_to_file[image_id]))
    display.display(display.Image(train_id_to_file[train_ids[closest_idx]]))

# 5. Training a Multi-Class Classification Network

We now build a two layer classification network, which takes 4096-dimensional vectors as input and outputs the probabilities of the 80 categories present in MSCOCO. 

For this purpose, we utilize two layers (both containing sigmoid activation functions) with the hidden dimension set to 512. 

In [ ]:
# First we construct a class for the model

class MultiClassClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        When constructing the model, we initialize two linear modules and assign them
        as class fields.
        """
        super(MultiClassClassifier, self).__init__()
        self.layer1 = torch.nn.Linear(input_size, hidden_size)
        self.layer2 = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        """
        Pass the input through the network, applying the sigmoid activation function after each layer.
        """
        return self.layer2(torch.sigmoid(self.layer1(x)))



In [ ]:
model_c = MultiClassClassifier(4096, 512, 80).cuda()

In [ ]:
import pickle
training_vectors = pickle.load(open("outputs/training_vectors.p", "rb"))
validation_vectors = np.load("outputs/validation_vectors")

In [ ]:
# Randomize training vectors
# training_vectors = np.random.random((len(train_ids), 4096))

# Now we prepare the input data, by converting the generated vectors into PyTorch variables.
training_input = [Variable(torch.FloatTensor(train_vector)).cuda() for train_vector,train_id in zip(training_vectors,train_ids) if len(list(train_id_to_categories[train_id])) > 0]

# Construct the validation input
with torch.no_grad():
    validation_input = [Variable(torch.FloatTensor(val_vector)).cuda() for val_vector in validation_vectors]
#     validation_input = [Variable(torch.FloatTensor(val_vector), volatile=True).cuda() for val_vector in validation_vectors]

# The output data is prepared by representing each output as a binary vector of categories
training_output = []
for i in range(len(train_ids)):
    categories = list(train_id_to_categories[train_ids[i]])
    if len(categories) == 0:
        continue
  
    training_vector = np.zeros(len(category_to_idx))
    indices = [category_to_idx[category] for category in categories]
    training_vector[indices] = 1
    training_output.append(training_vector)

training_output = Variable(torch.FloatTensor(training_output)).cuda()

In [ ]:
# The output data is prepared by representing each output as a binary vector of categories
validation_output = []
for i in range(len(val_ids)):
    categories = list(val_id_to_categories[val_ids[i]])  
    training_vector = np.zeros(len(category_to_idx))
    indices = [category_to_idx[category] for category in categories]
    training_vector[indices] = 1
    validation_output.append(training_vector)
with torch.no_grad():
    validation_output = Variable(torch.FloatTensor(validation_output)).cuda()
#     validation_output = Variable(torch.FloatTensor(validation_output), volatile=True).cuda()

In [ ]:
def validate(model):
    """
    Given a model, return the validation loss.
    """
    criterion = nn.MultiLabelSoftMarginLoss()

    # Create the input/output for the model
    x = torch.stack(validation_input)
    y = validation_output

    # Run it through the prediction
    y_pred = model(x)
        
    # Compute and return loss
    loss = criterion(y_pred, y)
    return loss.item()

In [ ]:
def train(model, learning_rate=0.0001, batch_size=100, epochs=10):
    """
    Training function which takes as input a model, a learning rate and a batch size.
  
    After completing a full pass over the data, the function exists, and the input model will be trained.
    """
    # Define the criterion and optimizer.
    criterion = nn.MultiLabelSoftMarginLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Keep track of the losses, for the purposes of plotting.
    train_losses = []
    val_losses = []

    # Determine number of minibatches
    
    num_iter = epochs * len(training_input)//batch_size 
    print(num_iter)
    for i in range(num_iter):
        start_idx = i * batch_size % len(training_input)

        # Retrieve the next batch of training data.
        x = torch.stack(training_input[start_idx:start_idx+batch_size])
        y = training_output[start_idx:start_idx+batch_size]
        
        # Forward pass
        y_pred = model(x)

        # Compute and print loss
        loss = criterion(y_pred, y)
        if i % 100 == 0:
            train_losses.append(loss.item())
            val_losses.append(validate(model))
            print(i, train_losses[-1], val_losses[-1])

        # Zero gradients, perform backwards pass and update model weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  

    return train_losses, val_losses    

# Finally train the model
train(model_c)

In [ ]:
# Now we repeat step two using the two layer classifier.
softmax = nn.Softmax(dim=1)
model_c.eval()
for i,image_id in enumerate(val_ids):
    # Display the image.
    display.display(display.Image(val_id_to_file[image_id]))

    # Print all of the category labels for this image.
    print("Ground Truth Labels:")
    for i,category in enumerate(val_id_to_categories[image_id]):
        print("%d. %s" % (i, category_to_name[category]))
  
    # Run the image through the model and softmax.
    label_likelihoods = torch.sigmoid(model_c(validation_input[i].unsqueeze(0))).squeeze()

    # Get the top 5 labels, and their corresponding likelihoods.
    probs, indices = label_likelihoods.topk(5)

    # Iterate and print out the predictions.
    print("Predictions:")
    for i in range(5):
        print("%d. %s (%.3f)" % (i, category_idx_to_name[indices.data[i]], probs.data[i]))

# 6. End-to-End Model Fine-tuning

Instead of training *only* the final two layers, we now create an end-to-end model and train the entire thing. 

In [ ]:
model_c.load_state_dict(torch.load("./model_c.pth"))

In [ ]:
# First we construct a class for the model
class EndToEndModel(torch.nn.Module):
    def __init__(self, vgg_model, classifier_c):
        """
        When constructing the model, we initialize two linear modules and assign them
        as class fields. We also, as done earlier, remove the final layer of the vgg model.
        """
        super(EndToEndModel, self).__init__()
        self.features = vgg_model.features
        self.classifier = nn.Sequential(*list(vgg_model.classifier.children())[:-1])
        self.classifier_c = classifier_c

    
    def forward(self, x):
        """
        Pass the input through the network, applying the sigmoid activation function after each layer.
        """
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return self.classifier_c(x)

model = EndToEndModel(vgg_model,model_c).cuda()

In [ ]:
def validate(model):
    """
    Given a model, return the validation loss.
    """
    # Now we prepare the input data, by converting the generated vectors into PyTorch variables.
    with torch.no_grad():
        validation_input = [load_image(val_id_to_file[val_id]).squeeze() for val_id in val_ids[:50]]


    criterion = nn.MultiLabelSoftMarginLoss()

    # Create the input/output for the model
    x = torch.stack(validation_input)
    y = validation_output[:50]

    # Run it through the prediction
    y_pred = model(x)
        
    # Compute and return loss
    loss = criterion(y_pred, y)
    return loss.item()

def create_training(start, end):
    training_input = [load_image(train_id_to_file[train_id]).squeeze() for train_id in train_ids[start:end]
                      if len(list(train_id_to_categories[train_id])) > 0]

    # The output data is prepared by representing each output as a binary vector of categories
    training_output = []
    for i in range(start,min(len(train_ids),end)):
        categories = list(train_id_to_categories[train_ids[i]])
        if len(categories) == 0:
            continue

        training_vector = np.zeros(len(category_to_idx))
        indices = [category_to_idx[category] for category in categories]
        training_vector[indices] = 1
        training_output.append(training_vector)
    training_output = Variable(torch.FloatTensor(training_output)).cuda()
    
    return training_input, training_output

def train(model, learning_rate=0.00001, batch_size=20, epochs=1):
    """
    Training function which takes as input a model, a learning rate and a batch size.
  
    After completing a full pass over the data, the function exists, and the input model will be trained.
    """
    # Define the criterion and optimizer.
    model.train()
    criterion = nn.MultiLabelSoftMarginLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Keep track of the losses, for the purposes of plotting.
    train_losses = []
    val_losses = []

    # Determine number of minibatches
    num_iter = epochs * len(train_id_to_file)//batch_size 
    for i in range(num_iter):
        print("Starting iteration: ", i)
        
        start_idx = i * batch_size % len(train_id_to_file)
        
        training_input, training_output = create_training(start_idx, start_idx + batch_size)

        # Retrieve the next batch of training data.
        x = torch.stack(training_input)
        y = training_output

        # Forward pass
        y_pred = model(x)

        # Compute and print loss
        loss = criterion(y_pred, y)

        # Zero gradients, perform backwards pass and update model weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  
        
        del x,y,y_pred

        if i % 10 == 0:
            model.eval()
            train_losses.append(loss.item())
            val_losses.append(validate(model))
            print(i, train_losses[-1], val_losses[-1])
            model.train()
        
    return train_losses, val_losses    

# Finally train the model
train(model)

In [ ]:
# Now we repeat step two using the end-to-end classifier.
model.eval()
with torch.no_grad():
    validation_input = [load_image(val_id_to_file[val_id]).squeeze() for val_id in val_ids[:50]]
softmax = nn.Softmax(dim=1)
for i,image_id in enumerate(val_ids):
    # Display the image.
    display.display(display.Image(val_id_to_file[image_id]))

    # Print all of the category labels for this image.
    print("Ground Truth Labels:")
    for i,category in enumerate(val_id_to_categories[image_id]):
        print("%d. %s" % (i, category_to_name[category]))
  
    # Run the image through the model and softmax.
    label_likelihoods = torch.sigmoid(model(validation_input[i].unsqueeze(0))).squeeze()

    # Get the top 5 labels, and their corresponding likelihoods.
    probs, indices = label_likelihoods.topk(5)

    # Iterate and print out the predictions.
    print("Predictions:")
    for i in range(5):
        print("%d. %s (%.3f)" % (i, category_idx_to_name[indices.data[i]], probs.data[i]))

# 7. Hyper-parameter Tuning

Now we do a grid search over the learning rate and batch size.

In [ ]:
best_params = None
best_loss = float('inf')
for learning_rate in [0.0001, 0.001, 0.01]:
    for batch_size in [50, 50, 5]:
        model = EndToEndModel(vgg_model, 4096, 512, 80).cuda()
        
        train_losses, val_losses = train(model, learning_rate=learning_rate, batch_size=batch_size)

        plt.plot(losses)
        plt.title('Training Losses (learning rate = %.3f, batch_size = %d' % (learning_rate, batch_size))
        plt.show()
    
        if val_losses[-1] < best_loss:
            best_loss = val_losses[-1]
            best_params = (learning_rate, batch_size) 